In [2]:
!pip install numpy
!pip install pandas
!pip install scikit-image
!pip install scikit-learn

Generation of Hog Feature from Training Set.

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score,precision_recall_fscore_support
df=pd.read_csv('input.csv')
df1=df[df['Usage']=='Training']
df2=df[(df['Usage']=='PublicTest') | (df['Usage']=='PrivateTest')]
hog_features=[]
hog_images=[]
for index,rows in df1.iterrows():
	s=rows['pixels'].split()
	r=np.array(s).astype('uint8')
	r=r.reshape(48,48)
	d,hog_image=hog(r,orientations=8,pixels_per_cell=(4,4),cells_per_block=(1,1),block_norm='L2',visualize=True)
	hog_images.append(hog_image)
	hog_features.append(d)

Generation of Hog Feature from Test Set.

In [0]:
test_hog_features=[]
test_hog_images=[]
for index,rows in df2.iterrows():
	s=rows['pixels'].split()
	r=np.array(s).astype('uint8')
	r=r.reshape(48,48)
	d,hog_image=hog(r,orientations=8,pixels_per_cell=(4,4),cells_per_block=(1,1),block_norm='L2',visualize=True)
	test_hog_images.append(hog_image)
	test_hog_features.append(d)

Generation of SVM Classifier Model using the Hog Features generated above and evaluation of training and test set based on model generated.

In [3]:
hog_features=np.array(hog_features)
test_hog_features=np.array(test_hog_features)
np.savetxt("train_hog_features.csv",hog_features,delimiter=",",header="pixels",comments='')
np.savetxt("test_hog_features.csv",test_hog_features,delimiter=",",header="pixels",comments='')
labels=df1['emotion (label)'].values.reshape(len(df1.index),1).ravel()
test_labels=df2['emotion (label)'].values.reshape(len(df2.index),1).ravel()
clf=svm.SVC()
clf.fit(hog_features,labels)
y_train_pred=clf.predict(hog_features)
y_pred=clf.predict(test_hog_features)
np.save('PredictedLabels',y_pred)
print("Training Evaluation")
print()
print(classification_report(labels, y_train_pred))
precision,recall,f1,_=precision_recall_fscore_support(labels, y_train_pred,average='weighted')
print("Precision"+":"+" "+str(precision))
print("Recall"+":"+" "+str(recall))
print("F1"+":"+" "+str(f1))
print()
print("Testing Evaluation")
print()
print(classification_report(test_labels, y_pred))
precision,recall,f1,_=precision_recall_fscore_support(test_labels, y_pred,average='weighted')
print("Precision"+":"+" "+str(precision))
print("Recall"+":"+" "+str(recall))
print("F1"+":"+" "+str(f1))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Training Evaluation

              precision    recall  f1-score   support

           0       0.83      0.97      0.90     21494
           1       0.83      0.41      0.55      7215

    accuracy                           0.83     28709
   macro avg       0.83      0.69      0.72     28709
weighted avg       0.83      0.83      0.81     28709

Precision: 0.8297181867828234
Recall: 0.8304016162179108
F1: 0.8086522555899983

Testing Evaluation

              precision    recall  f1-score   support

           0       0.83      0.97      0.89      5404
           1       0.80      0.41      0.54      1774

    accuracy                           0.83      7178
   macro avg       0.82      0.69      0.72      7178
weighted avg       0.82      0.83      0.81      7178

Precision: 0.8244158716594597
Recall: 0.8282251323488437
F1: 0.8065426433113733


Storing the Predicted Labels and corresponding Row Indices in input file in csv format.

In [4]:
a=np.load('PredictedLabels.npy')
b=a.tolist()
m=df.index[(df['Usage']=='PublicTest') | (df['Usage']=='PrivateTest')]
m=m.tolist()
r=[]
for i in range(len(a)):
	r.append([a[i],m[i]])
r=np.array(r)
print(r)
np.savetxt("Predicted_Test_Labels.csv",r,delimiter=",",header="Label,Row_Index",comments='')

[[    0 28709]
 [    0 28710]
 [    0 28711]
 ...
 [    0 35884]
 [    0 35885]
 [    0 35886]]
